In [350]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pyplot
import os

año = 2022

rutas = {
    'tee': { 
        'dir': '/mnt/c/Users/aaria/Documents/informe/TEE/', 
        'hoja': 'BALANCE TOTAL', 
        'dimension': 'A', 
        'fila_inicio': 12,
        'rango_columnas': 'D:L'
    },
    'tee centrales': {
        'dir': '/mnt/c/Users/aaria/Documents/informe/TEE/',
        'hoja': 'BALANCE FISICO TD', 
        'dimension': 'A', 
        'fila_inicio': 2,
        'rango_columnas': 'O:U'
    },
    'tep': {
        'dir': '/mnt/c/Users/aaria/Documents/informe/TEP/',
        'hoja': 'BALANCE',
        'dimension': 'A',
        'fila_inicio': 15,
        'rango_columnas': 'C:J'
    },
    'tdc': {
        'dir': '/mnt/c/Users/aaria/Documents/informe/TDC/',
        'hoja': 'BALANCE',
        'dimension': 'A',
        'fila_inicio': 21,
        'rango_columnas': 'C:J'
    },
    'crf': {
        'dir': '/mnt/c/Users/aaria/Documents/informe/CRF/',
        'hoja': ['Resumen', 'Resumen Compensaciones', 'Resumen Cargos'],
        'dimension': {
            'Resumen': 'A',
            'Resumen Compensaciones': 'A',
            'Resumen Cargos': 'A'
        },
        'fila_inicio': {
            'Resumen': 11,
            'Resumen Compensaciones': 4,
            'Resumen Cargos': 4
        },
        'rango_columnas': {
            'Resumen': 'C:J',
            'Resumen Compensaciones': 'B:K',
            'Resumen Cargos': 'C:J'
        }
        
    },
    'compensacion_cdf': {
        'dir': '/mnt/c/Users/aaria/Documents/informe/CDF/',
        'hoja': 'RESUMEN',
        'dimension': 'A',
        'fila_inicio': 11,
        'rango_columnas': 'AY:BA'
    },
    'balance_cdf': {
        'dir': '/mnt/c/Users/aaria/Documents/informe/CDF/',
        'hoja': 'RESUMEN',
        'dimension': 'B',
        'fila_inicio': 11,
        'rango_columnas': 'AL:AT'
    },
    'cdo': {
        'dir': '/mnt/c/Users/aaria/Documents/informe/CDO/',
        'hoja': 'RESUMEN 1',
        'dimension': 'A',
        'fila_inicio': 10,
        'rango_columnas': 'Z:AJ'
    },
    'cmg': {
        'dir': '/mnt/c/Users/aaria/Documents/informe/CMG/',
        'hoja': '',
        'dimension': '',
        'fila_inicio': 0,
        'rango_columnas': ''
    }
}

meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

In [ ]:
#GRAFICAS

#FORMATO DE TABLAS
def formato_tabla (tabla, valorizada):  
    if valorizada:
        for col in tabla.columns:
            if tabla[col].dtype == 'float64':
                tabla[col] = pd.Series(tabla[col]).round(2).fillna(value='')
            elif tabla[col].dtype == 'object':
                tabla[col] = pd.Series(tabla[col]).fillna(value='')
        return tabla
    else:
        for col in tabla.columns:
            if tabla[col].dtype == 'float64':
                if tabla[col].max() > 10000 or tabla[col].min() < -10000:
                    tabla[col] = pd.Series(tabla[col])/1000000
                tabla[col] = pd.Series(tabla[col]).fillna(value='')
            elif tabla[col].dtype == 'object':
                tabla[col] = pd.Series(tabla[col]).fillna(value='')
        return tabla
    
def formato_tabla_energia (tabla, valorizada):
    tabla = formato_tabla(tabla, valorizada)
    
    tabla.rename(columns={
        'INYECCIONES': 'INYECCIONES FÍSICAS', 
        'RETIROS': 'RETIROS FÍSICOS', 
        'INYECCIONES.1': 'INYECCIONES POR CONTRATO', 
        'RETIROS.1': 'RETIROS POR CONTRATO'
    }, inplace=True)
    
    return tabla

def formato_tabla_potencia (tabla, valorizada):
    tabla = formato_tabla(tabla, valorizada)
    
    tabla.rename(columns={
        tabla.columns[2]: 'INYECCIONES POR CONTRATO',
        tabla.columns[3]: 'INYECCIONES FÍSICAS',
        tabla.columns[4]: 'RETIROS POR CONTRATO',
        tabla.columns[5]: 'RETIROS FÍSICOS',
        tabla.columns[6]: 'EXCEDENTE DE POTENCIA DE PUNTA',
        tabla.columns[7]: 'DEFICIT DE POTENCIA DE PUNTA'
    }, inplace=True)
    
    return tabla

def formato_tabla_dc (tabla, valorizada):
    tabla = formato_tabla(tabla, valorizada)
    
    tabla.rename(columns={
        tabla.columns[2]: 'INYECCIONES POR CONTRATO',
        tabla.columns[3]: 'INYECCIONES FÍSICAS',
        tabla.columns[4]: 'RETIROS POR CONTRATO',
        tabla.columns[5]: 'RETIROS FÍSICOS',
        tabla.columns[6]: 'BALANCE ACREEDORES',
        tabla.columns[7]: 'BALANCE DEUDORES'
    }, inplace=True)
    
    return tabla

#EXTRACCIÓN DE DATOS
def leer_tablas (ruta, posicion, dos_tablas, posicion2 = 0):
    archivos = os.listdir(ruta[0])
    data = []
    i = 0
    
    for archivo in archivos:
        if dos_tablas:
            data.append(
                pd.read_excel(
                    ruta[0] + archivo,
                    sheet_name = ruta[1],
                    header = posicion[1][i] + posicion2,
                    nrows = posicion[1][i],
                    usecols = posicion[2]
                ).assign(MES = meses[i])
            )
        
        else:
            data.append(
                pd.read_excel(
                    ruta[0] + archivo,
                    sheet_name = ruta[1],
                    header = posicion[0],
                    nrows = posicion[1][i],
                    usecols = posicion[2]
                ).assign(MES = meses[i])
            )

        i += 1
    
    for mes in meses[len(archivos):]:
        datos = pd.DataFrame()
        data.append(datos.reindex_like(data[0]).assign(MES = mes))
        
    tabla = pd.concat([d for d in data])
    
    return tabla

def posiciones_tablas (ruta, hoja = ''):
    archivos = os.listdir(ruta['dir'])
    dimensiones = []
    
    if hoja == '':     
        for archivo in archivos:
            dimensiones.append(
                pd.read_excel(
                    ruta['dir'] + archivo,
                    sheet_name = ruta['hoja'],
                    usecols = ruta['dimension'],
                    skiprows = 0,
                    nrows = 1,
                    header = None,
                    names = ['valor']
                ).iloc[0]['valor'])

        return [ruta['fila_inicio'], dimensiones, ruta['rango_columnas']]
    else:
        for archivo in archivos:
            dimensiones.append(
                pd.read_excel(
                    ruta['dir'] + archivo,
                    sheet_name = hoja,
                    usecols = ruta['dimension'][hoja],
                    skiprows = 0,
                    nrows = 1,
                    header = None,
                    names = ['valor']
                ).iloc[0]['valor'])
            
        return [ruta['fila_inicio'][hoja], dimensiones, ruta['rango_columnas'][hoja]]    

In [378]:
#ENERGÍA

#DATOS DE ENERGÍA
ruta_energia = [rutas['tee']['dir'], rutas['tee']['hoja']]
posiciones_energia = posiciones_tablas(rutas['tee'])

ruta_centrales = [rutas['tee centrales']['dir'], rutas['tee centrales']['hoja']]
posiciones_centrales = posiciones_tablas(rutas['tee centrales'])

#BALANCE FÍSICO
energia_fisica = leer_tablas(ruta_energia, posiciones_energia, False)
energia_fisica = formato_tabla_energia(energia_fisica, False)

#BALANCE VALORIZADO
energia_valorizada = leer_tablas(ruta_energia, posiciones_energia, True, 23)
energia_valorizada = formato_tabla_energia(energia_valorizada, True)

#INYECCIONES X CENTRALES
energia_centrales = leer_tablas(ruta_centrales, posiciones_centrales, False)
energia_centrales = formato_tabla_energia(energia_centrales, False)

In [379]:
#TABLAS DE ENERGÍA

#TABLA 3
energia_fisica['INYECCIONES FÍSICAS'] = energia_fisica['INYECCIONES FÍSICAS'].replace(to_replace = '', value = 0)
energia_fisica_inyecciones = np.round(pd.pivot_table(energia_fisica, values = 'INYECCIONES FÍSICAS',
                                                columns = ['MES'],
                                                index = 'EMPRESA'), 3).fillna(0)
energia_fisica_inyecciones = energia_fisica_inyecciones[meses]
energia_fisica_inyecciones['Total'] = energia_fisica_inyecciones.sum(axis = 1)
energia_fisica_inyecciones = energia_fisica_inyecciones.iloc[1: , :]
energia_fisica_inyecciones = energia_fisica_inyecciones.replace(to_replace = 0, value = '').style.set_table_styles(
     [{
           'selector': 'th',
           'props': [
               ('text-align', 'center')]
       }]
).set_properties(
    subset = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'], 
    **{'width': '55px'}
).format(precision = 3)

#TABLA 4
energia_fisica['RETIROS FÍSICOS'] = energia_fisica['RETIROS FÍSICOS'].replace(to_replace = '', value = 0)
energia_fisica_retiros = np.round(pd.pivot_table(energia_fisica, values = 'RETIROS FÍSICOS',
                                                columns = ['MES'],
                                                index = 'EMPRESA'), 3).fillna(0)
energia_fisica_retiros = energia_fisica_retiros[meses]
energia_fisica_retiros['Total'] = energia_fisica_retiros.sum(axis = 1)
energia_fisica_retiros = energia_fisica_retiros.iloc[1: , :]
energia_fisica_retiros = energia_fisica_retiros.replace(to_replace = 0, value = '').style.set_table_styles(
     [{
           'selector': 'th',
           'props': [
               ('text-align', 'center')]
       }]
).set_properties(
    subset = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'], 
    **{'width': '55px'}
).format(precision = 3)

#TABLA 5
energia_centrales['TOTALES'] = energia_centrales['TOTALES'].replace(to_replace = '', value = 0)
energia_centrales = np.round(pd.pivot_table(energia_centrales, values = 'TOTALES',
                                                columns = ['MES'],
                                                index = [' AGENTE', 'CENTRALES']), 3).fillna(0)
energia_centrales = energia_centrales[meses]
energia_centrales['Total'] = energia_centrales.sum(axis = 1)
energia_centrales = energia_centrales.iloc[1: , :]
energia_centrales = energia_centrales.replace(to_replace = 0, value = '').style.set_table_styles(
     [{
           'selector': 'th',
           'props': [
               ('text-align', 'center')]
       }]
).set_properties(
    subset = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'], 
    **{'width': '55px'}
).format(thousands = ',', precision = 2)

In [ ]:
#POTENCIA

#DATOS DE POTENCIA
ruta_potencia = [rutas['tep']['dir'], rutas['tep']['hoja']]
posiciones_potencia = posiciones_tablas(rutas['tep'])

#BALANCE FÍSICO
potencia_fisica = leer_tablas(ruta_potencia, posiciones_potencia, False)
formato_tabla_potencia(potencia_fisica, False)
potencia_fisica = potencia_fisica.style.format(precision = 3).hide(axis = "index")

#BALANCE VALORIZADO
potencia_valorizada = leer_tablas(ruta_potencia, posiciones_potencia, True, 28)
formato_tabla_potencia(potencia_valorizada, True)
potencia_valorizada = potencia_valorizada.style.format(thousands = ',', precision = 2).hide(axis = 'index')

In [ ]:
#DERECHO DE CONEXIÓN
resolucion_tdc = 'SIE-118-2021-PJ'

#DATOS DE DERECHO DE CONEXIÓN
ruta_tdc = [rutas['tdc']['dir'], rutas['tdc']['hoja']]
posiciones_tdc = posiciones_tablas(rutas['tdc'])

#BALANCE DC
tdc = leer_tablas(ruta_tdc, posiciones_tdc, True, 20)
formato_tabla_dc(tdc, True)
tdc = tdc.style.format(thousands = ',', precision = 2).hide(axis = 'index')

In [ ]:
#TABLAS DE REGULACIÓN DE FRECUENCIA
resolucion_crf = 'SIE-120-2021-MEM'

#DATOS REGULACIÓN DE FRECUENCIA
ruta_crf_compensaciones = [rutas['crf']['dir'], rutas['crf']['hoja'][0]]
ruta_crf_cargos = [rutas['crf']['dir'], rutas['crf']['hoja'][1]]
ruta_crf_balance = [rutas['crf']['dir'], rutas['crf']['hoja'][2]]

posiciones_crf_compensaciones = posiciones_tablas(rutas['crf'], rutas['crf']['hoja'][0])
posiciones_crf_cargos = posiciones_tablas(rutas['crf'], rutas['crf']['hoja'][1])
posiciones_crf_balance = posiciones_tablas(rutas['crf'], rutas['crf']['hoja'][2])

#COMPENSACIONES CRF
crf_compensaciones = leer_tablas(ruta_crf_compensaciones, posiciones_crf_compensaciones, False)
formato_tabla(crf_compensaciones, True)
crf_compensaciones = crf_compensaciones.style.format(thousands = ',', precision = 2).hide(axis = 'index')

#CARGOS CRF
crf_cargos = leer_tablas(ruta_crf_cargos, posiciones_crf_cargos, False)
formato_tabla(crf_cargos, True)
crf_cargos = crf_cargos.style.format(thousands = ',', precision = 2).hide(axis = 'index')

#BALANCE CRF
crf_balance = leer_tablas(ruta_crf_balance, posiciones_crf_balance, False)
formato_tabla(crf_balance, True)
crf_balance = crf_balance.style.format(thousands = ',', precision = 2).hide(axis = 'index')

In [ ]:
#TABLAS DE DESPACHO FORZADO
resolucion_cdf = 'SIE-119-2021-MEM'

#DATOS DESPACHO FORZADO
ruta_compensacion_cdf = [rutas['compensacion_cdf']['dir'], rutas['compensacion_cdf']['hoja']]
posiciones_compensacion_cdf = posiciones_tablas(rutas['compensacion_cdf'])

ruta_balance_cdf = [rutas['balance_cdf']['dir'], rutas['balance_cdf']['hoja']]
posiciones_balance_cdf = posiciones_tablas(rutas['balance_cdf'])

#COMPENSACIÓN CDF
compensacion_cdf = leer_tablas(ruta_compensacion_cdf, posiciones_compensacion_cdf, False)
formato_tabla(compensacion_cdf, True)
compensacion_cdf = compensacion_cdf.style.format(thousands = ',', precision = 2).hide(axis = 'index')

#BALANCE CDF
balance_cdf = leer_tablas(ruta_balance_cdf, posiciones_balance_cdf, False)
formato_tabla(balance_cdf, True)
balance_cdf = balance_cdf.style.format(thousands = ',', precision = 2).hide(axis = 'index')

In [ ]:
#TABLAS DE DESVÍO DE LA OPERACIÓN
resoluciones_cdo = ['SIE 374-2012', 'SIE-018-2013-MEM', 'SIE-041-2013-MEM']

#DATO DESVÍO DE LA OPERACIÓN
ruta_cdo = [rutas['cdo']['dir'], rutas['cdo']['hoja']]
posiciones_cdo = posiciones_tablas(rutas['cdo'])

#BALANCE CDO
cdo = leer_tablas(ruta_cdo, posiciones_cdo, False)
formato_tabla(cdo, True)
cdo = cdo.style.format(thousands = ',', precision = 2).hide(axis = 'index')

In [ ]:
#TABLAS DE COSTOS MARGINALES


In [ ]:
#GRÁFICAS


# ÍNDICE
1. [INTRODUCCIÓN](#introduccion)

# INTRODUCCIÓN <a name="introduccion" />

<div style="text-align: justify">
El presente informe contiene un resumen estadístico de las inyecciones y retiros de los generadores, los retiros de los distribuidores y de los Usuarios No Regulados, así como las correspondientes Transacciones Económicas en el Mercado Eléctrico Mayorista, destacando los pagos al propietario del sistema de transmisión, los pagos realizados por concepto de servicios de regulación de frecuencia y los pagos resultantes de la aplicación del mecanismo de compensación de unidades generadoras por despacho forzado y compensación por desvío, conforme a lo establecido en la Resolución SIE-119-2021-MEM y en la Resolución SIE-041-2013-MEM, respectivamente.
</div>

## 1. RESUMEN EJECUTIVO TRANSACCIONES ECONÓMICAS
<!--
<div style="text-align: justify">
En el año 2022:
<ul>
    <li>El promedio de los Costos Marginales de Energía Activa de Corto Plazo para el sistema principal alcanzó el valor {{cmg_promedio}}, representando {{cambio_cmg_año_anterior}} de {{cambio_cmg_año_anterior_porcentaje}} con relación al promedio del año anterior para el mismo mes ({{cmg_año_anterior}}) y {{cambio_cmg_mes_anterior}} de {{cambio_cmg_mes_anterior_porcentaje}} con relación al mes anterior ({{cmg_mes_anterior}}). El Costo Marginal de Potencia de Punta fue {{cmg_potencia}}, y el Derecho de Conexión Unitario fue de {{dc_unitario}}.</li>
</div>
-->

## 2. OBJETIVO

## 3. TRANSACCIONES ECONÓMICAS DE ENERGÍA
### 3.1 Balance de Energía de los Agentes del MEM
<div style="align: center;">
    {{balance_energia}}
</div>

## 4. TRANSACCIONES PROVISIONALES DE POTENCIA

## 5. PEAJE DE TRANSMISIÓN

## 6. SERVICIO DE REGULACIÓN DE FRECUENCIA

## 7. COMPENSACIÓN POR DESPACHO FORZADO SEGÚN RESOLUCIÓN {{resolucion_cdf}}

## 8. COMPENSACIÓN POR DESVÍO SEGÚN RESOLUCIONES {{resoluciones_cdo[0]}}, {{resoluciones_cdo[1]}} Y {{resoluciones_cdo[2]}}

## 9. RESUMEN DE LAS TRANSACCIONES DEL MEM EN EL {{año}}

## 10. COSTOS MARGINALES DE CORTO PLAZO DE ENERGÍA

## 11. INDICADORES DEL MEM

## 12. ASPECTOS RELEVANTES